In [1]:
library(data.table)

In [2]:
dt <- fread('out.I.tsv')

In [3]:
head(dt)

,V1,V2,V3,V4,V5,V6,V7
1,H5JCKCCXX:7:10:724696:0/1,C*01:20,28,16,16,1,MKYFFTSVSRPGRGEP
2,H5JCKCCXX:7:10:724696:0/1,C*01:58,28,16,16,1,MKYFFTSVSRPGRGEP
3,H5JCKCCXX:7:10:724696:0/1,C*01:86N,28,16,16,1,MKYFFTSVSRPGRGEP
4,H5JCKCCXX:7:10:724696:0/1,C*01:22,28,16,16,1,MKYFFTSVSRPGRGEP
5,H5JCKCCXX:7:10:724696:0/1,C*01:13,28,16,16,1,MKYFFTSVSRPGRGEP
6,H5JCKCCXX:7:10:724696:0/1,C*01:40,28,16,16,1,MKYFFTSVSRPGRGEP


In [4]:
setnames(dt, c('q', 't', 'pos', 'len', 'align', 'specific', 'seq'))

In [5]:
nrow(dt)

[1] 408939

In [6]:
tt <- dt[, qp := sub('/\\d$', '', q)]
nr <- dt[, .(pair = length(unique(q))), keyby = qp]
setkey(dt, qp)
dt <- nr[dt]
head(dt)

,qp,pair,q,t,pos,len,align,specific,seq
1,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*15:58,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
2,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*40:10,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
3,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*40:91,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
4,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*50:38,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
5,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*27:17,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
6,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*15:70,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP


In [7]:
nr <- dt[, .(pair2 = length(unique(q))), keyby = .(qp, t)]
setkey(dt, qp, t)
dt <- nr[dt]
head(dt)

,qp,t,pair2,pair,q,pos,len,align,specific,seq
1,H5JCKCCXX:1:10:149660:0,B*07:02,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
2,H5JCKCCXX:1:10:149660:0,B*07:03,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
3,H5JCKCCXX:1:10:149660:0,B*07:04,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
4,H5JCKCCXX:1:10:149660:0,B*07:05,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
5,H5JCKCCXX:1:10:149660:0,B*07:06,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
6,H5JCKCCXX:1:10:149660:0,B*07:07,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,1,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP


In [8]:
dt <- dt[pair == pair2]
nrow(dt)

[1] 371291

In [10]:
library(IRanges)

In [11]:
dt <- dt[grepl('\\d$', t) & align / len > 0.8]
setkey(dt, t)
cov <- dt[, .(
    n0 = .N, 
    n1 = sum(specific), 
    cov0 = sum(width(reduce(IRanges(pos, width = len)))),
    cov1 = sum(width(reduce(IRanges(pos[specific == 1], width = len[specific == 1]))))
), keyby = t]

In [60]:
real <- c('A*02:01', 'A*03:01', 'B*13:02', 'B*51:01', 'C*01:02', 'C*06:02', 'DRB1*07:01', 'DRB1*11:01', 'DQB1*02:AB', 'DQB1*03:01', 'DPB1*02:01', 'DPB1*02:01')

In [68]:
cand <- cov[, t[order(-cov1)][1:100], by = sub('\\*.+', '', t)]
sum(real %in% cand$V1)

[1] 6

In [17]:
pure <- dt[specific == 1]
nonpure <- dt[specific == 0]

In [18]:
nrow(pure)

[1] 202255

In [19]:
nrow(nonpure)

[1] 3353

In [69]:
good <- dt[align / len > 0.8]
nrow(good)

[1] 205608

In [123]:
tt <- good[, v := ifelse(specific, 1, 0.1)]
mat <- dcast(good, q ~ t, value.var = 'v', fill = 0)

In [124]:
tt <- mat[, q := NULL]

In [125]:
tail(sort(colSums(mat)))

A*03:196 A*03:134  A*03:20  A*03:26  A*03:89  A*03:01 
   163.2    167.1    169.2    174.2    175.2    176.2

In [126]:
temp <- sort(colSums(mat))

In [152]:
tail(temp[grepl('C\\*', names(temp))], 20)

C*06:111 C*06:105  C*06:02  C*06:12 C*06:129 C*06:153  C*06:24  C*06:83 
    98.1     99.2    103.2    103.2    103.2    103.2    103.2    103.2 
 C*06:93  C*06:06  C*06:09 C*06:138  C*06:55  C*06:56 C*06:100  C*06:30 
   103.2    104.2    104.2    105.2    105.2    106.2    111.1    114.2 
C*06:108 C*06:109 C*06:110  C*06:73 
   120.2    120.2    120.2    127.2

In [130]:
maxScore <- sum(apply(mat, 1, max))
is.specific <- apply(mat, 1, max) == 1
sum(is.specific)

[1] 565

In [143]:
mat[, sum(pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:01`, `C*01:02`, `C*06:02`))]
mat[pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:01`, `C*01:02`, `C*06:02`) == 0 & is.specific, .N] 

[1] 493.3

[1] 72

In [153]:
mat[, sum(pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:01`, `C*01:11`, `C*06:109`))]
mat[pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:01`, `C*01:11`, `C*06:109`) == 0 & is.specific, .N] 

[1] 508.3

[1] 57

In [144]:
mat[, sum(pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:01`, `C*01:47`, `C*06:02`))]
mat[pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:01`, `C*01:47`, `C*06:02`) == 0 & is.specific, .N] 

[1] 503.3

[1] 62

In [133]:
mat[, sum(pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:30`, `C*01:02`, `C*06:02`))]
mat[pmax(`A*02:01`, `A*03:01`, `B*13:02`, `B*51:30`, `C*01:02`, `C*06:02`) == 0 & is.specific, .N]

[1] 489.3

[1] 76

In [142]:
real

[1] "A*02:01" "A*03:01" "B*13:02" "B*51:01" "C*01:02" "C*06:02"

In [154]:
allele.names <- colnames(mat)
allele.genes <- unique(sub('\\*.+', '', allele.names))
alleles <- 1:ncol(mat)
reads <- 1:nrow(mat)
na <- length(alleles)
nr <- length(reads)

In [156]:
library(lpSolve)
f.obj <- c(rep(0, na), ifelse(is.specific, 1, 0.1))
# constraints for hit incidence matrix
f.con <- do.call(rbind, mclapply(1:nr, function(i){
    con <- c(test[i,], rep(0, nr))
    con[na + i] <- -1
    con
}))
f.dir <- rep('>=', nr)
f.rhs <- rep(0, nr)
# constraints for 1 or 2 alleles per gene

f.con2 <- do.call(rbind, mclapply(allele.genes, function(gene){
    con <- c(rep(0, na), rep(0, nr))
    con[grep(sprintf("^%s", gene), allele.names)] <- 1
    rbind(con, con)
}))
f.dir2 <- rep(c('>=', '<='), length(allele.genes))
f.rhs2 <- rep(c( 1,    2  ), length(allele.genes))

# final constraints
f.con <- rbind(f.con, f.con2)
f.dir <- c(f.dir, f.dir2)
f.rhs <- c(f.rhs, f.rhs2)

In [157]:
system.time(lps <- lp('max', f.obj, f.con, f.dir, f.rhs, all.bin = T))

   user  system elapsed 
  1.234   0.056   1.290 

In [158]:
lps

Success: the objective function is 508 

In [159]:
solution <- lps$solution[alleles]
names(solution) <- allele.names
solution <- solution[order(-solution)]
solution <- solution[solution > 0]
solution

A*02:01  A*03:01  B*13:02  B*51:01  C*01:11 C*06:109 
       1        1        1        1        1        1

#TODO
- why C all wrong? reads from pseudo gene? non-unique reads?
- Add J/H/E etc
- Add bootstraping with reads (random drop reads)
- add bootstraping with alleles (remove one of the solution allele each time)
- consider cutting the MSA to fit common exon
- adjust objective function for position aware (say on the same MSA position, if there are 3 types of different sequences, then one of them must be leaky match)
- Adapt Class II
- Add homozygous penalty term

# C*01:02 vs C*01:11 pair
Two differences casued the problem:
- one difference at exon boundaries was lost in the mapping: A|AA vs G|AA, no reads able to map to this amino acid, so not possible to distinguish the two HLA types
- Another exon bondary: A|GG vs A|xx, reads no able to map to the 01:02 due to exon boundary, but able to map to A|xx (not at full read length), because the intron part happend to lead to matching amino acid

Therefore, **need to do mapping at exon level, and then check half codon after alignment**